In [6]:
from langchain_community.agent_toolkits.github.toolkit import GitHubToolkit
from langchain_community.utilities.github import GitHubAPIWrapper
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

from dotenv import load_dotenv

load_dotenv()

# Instantiate GitHub Toolkit
toolkit = GitHubToolkit.from_github_api_wrapper(GitHubAPIWrapper())
tools = [setattr(tool, "name", tool.mode) or tool for tool in toolkit.get_tools()]

llm = ChatOpenAI(model="gpt-4o", temperature=0)

agent_executor = create_react_agent(llm, tools)

def ai(query: str):
    # System prompt allows for broad range of instructions. But what does user input look like?
    events = agent_executor.stream(
        {"messages": [("system",
                       "You are a web server, serving HTML web pages. Do not use markdown for formatting. "
                       "All output must be in HTML format and will be displayed to an end user. "
                       "You need to start with doctype and html tags and provide the entire page"),
                      ("user", query)]},
        stream_mode="values",
    )
    data = ""
    for event in events:
        event["messages"][-1].pretty_print()
        data = event["messages"][-1]

    return data.content


In [7]:
from pathlib import Path

from dotenv import load_dotenv
from flask import Flask


# Load environment variables from .env
load_dotenv()

app = Flask(__name__)

@app.route('/', methods=['GET'])
def get_data():
    # Wow where is the logic?
    return ai(Path("my_app.txt").read_text())


## Goals
- Write natural language only
- Unstructured data
- Implicit algorithms
- ~~Dynamic tools~~
- ~~Human in the loop built-in~~
- ~~Consistent output~~
- ~~Iterative development~~

In [ ]:
app.run(debug=False, port=5001, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit


================================ Human Message =================================

Find the most prolific author of github issues, and I want to see all of their issues and comments

I will need to see ONLY the author, title, and comments. HTML elements can be styled using twitter bootstrap css.


================================== Ai Message ==================================
Tool Calls:
  get_issues (call_4SpkFKWa7GZcY9uKRVV9J59F)
 Call ID: call_4SpkFKWa7GZcY9uKRVV9J59F
  Args:
    no_input:
================================= Tool Message =================================
Name: get_issues

Found 2 issues:
[{'title': 'This is my second issue', 'number': 2, 'opened_by': 'yasonk'}, {'title': 'First issue here you go', 'number': 1, 'opened_by': 'yasonk'}]
================================== Ai Message ==================================
Tool Calls:
  get_issue (call_SELlIht6dRFNIJ8UzIx5dwFa)
 Call ID: call_SELlIht6dRFNIJ8UzIx5dwFa
  Args:
    issue_number: 2
  get_issue (call_AYzHh93W20vXC9o

127.0.0.1 - - [26/Feb/2025 22:53:54] "GET / HTTP/1.1" 200 -


================================== Ai Message ==================================

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>GitHub Issues by Most Prolific Author</title>
    <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">
</head>
<body>
    <div class="container mt-5">
        <h1 class="mb-4">GitHub Issues by Most Prolific Author</h1>
        <div class="card mb-3">
            <div class="card-header">
                <h5 class="card-title">Author: yasonk</h5>
            </div>
            <div class="card-body">
                <h6 class="card-subtitle mb-2 text-muted">Issue Title: This is my second issue</h6>
                <ul class="list-group">
                    <li class="list-group-item">Comment 1: Comment 1, because I'm building a community</li>
                    <li class="list-group-item">Comment 2: Comm

127.0.0.1 - - [26/Feb/2025 22:55:04] "GET / HTTP/1.1" 200 -


================================== Ai Message ==================================

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>GitHub Issues</title>
    <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">
</head>
<body>
    <div class="container mt-5">
        <h1 class="mb-4">GitHub Issues by yasonk</h1>
        
        <div class="accordion" id="issuesAccordion">
            <div class="card">
                <div class="card-header" id="headingOne">
                    <h2 class="mb-0">
                        <button class="btn btn-link" type="button" data-toggle="collapse" data-target="#collapseOne" aria-expanded="true" aria-controls="collapseOne">
                            First issue here you go
                        </button>
                    </h2>
                </div>

                <div id="collapseOne" cla